In [1]:
import sympy as sp
import minterpy as mp
import numpy as np
from minterpy.pointcloud_utils import *

from mpl_toolkits import mplot3d
%matplotlib inline

from mpl_toolkits.mplot3d import axes3d

import torch
import torchvision
from torchvision import transforms, datasets

import random
import numpy as np
import matplotlib.pyplot as plt

import os
from operator import itemgetter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import ot

from sklearn.neighbors import NearestNeighbors

import ripser
import persim
from persim import plot_diagrams

from operator import itemgetter


#do pip installs as follows to use vedo for plotting point clouds
#pip install vedo
#pip install ipyvtklink

import numpy as np
from vedo import *

import matplotlib.pyplot
from mpl_toolkits.mplot3d import Axes3D

import time
torch.set_printoptions(precision=8)


In [2]:
def _compute_distance_matrix(x, p=2):
    x_flat = x.view(x.size(0), -1)

    distances = torch.norm(x_flat[:, None] - x_flat, dim=2, p=p)

    return distances

def get_persistence_diagram(point_cloud, maximum_dim):

    point_cloud = torch.tensor(point_cloud)

    dist_matrix = _compute_distance_matrix(point_cloud, p=2)
    diagrams = ripser.ripser(dist_matrix.cpu().detach().numpy(), distance_matrix=True, maxdim=maximum_dim)['dgms']
    return diagrams, plot_diagrams(diagrams, show=True)


def random_sampling_from_pt_cloud(point_cloud, req_sample_size):
    
    point_cloud = torch.tensor(point_cloud)

    idx = np.random.randint(point_cloud.shape[0], size=req_sample_size)

    random_Sampled_point_cloud = point_cloud[idx,:]

    return random_Sampled_point_cloud

In [3]:
x, y, z = sp.symbols('x y z')

expr = x**2 + y**2 + z**2 - 1
poly = sp.Poly(expr, x, y, z)

# convert sympy polynomial to minyterpy polynomial
newt_poly = sympy_to_mp(poly, mp.NewtonPolynomial)


#sample points
point_data = sample_points_on_poly(1000,        # Number of points to be sampled
                                   newt_poly,  # Polynomial in Newton basis
                                   bounds=1, # Boundary of the Cubic domain to be sampled
                                   tol=1e-15)  # Tolerance in solution

In [221]:
# function to check whether the selected edge is going to close a potential loop

def expecting_a_cycle(actual_new_test, my_edge):

    left_ind = my_edge[0][0]
    right_ind = my_edge[0][1]
    found_right_ind = False
    going_nowhere= False

    new_test = actual_new_test
    actual_new_test_an = actual_new_test
    
    tracker = 0
    no_branches_formed = True
    loop_tracker = 0
    positions1 = (new_test == left_ind).nonzero(as_tuple=False)
    loops_collec = []
    current_loop = torch.tensor([])
    while (not(found_right_ind) or not(going_nowhere)):

        positions1 = (new_test == left_ind).nonzero(as_tuple=False)
        #print(new_test)
        #print()
        print('positions1.shape[0]',positions1.shape[0])
        #print()
        
        if(positions1.shape[0]>1):
            edge_to_delete = new_test[positions1[0][0]]
            no_branches_formed = False
            print('edge_to_delete first',edge_to_delete)
        branches_rising = positions1.shape[0]

        if(positions1.shape[0]==0):
            current_loop = torch.tensor([])
            #going_nowhere= True
            '''if(no_branches_formed):
                break'''
            
            left_ind = my_edge[0][0]

            deletable_edge_position1 = (actual_new_test == edge_to_delete[0]).nonzero(as_tuple=False)
            deletable_edge_position2 = (actual_new_test == edge_to_delete[1]).nonzero(as_tuple=False)

            deletable_edge_position1 = deletable_edge_position1[:,0]

            deletable_edge_position2 = deletable_edge_position2[:,0]

            a_cat_b1, counts1 = torch.cat([deletable_edge_position1, deletable_edge_position2]).unique(return_counts=True)
            deletable_row_position = a_cat_b1[torch.where(counts1.gt(1))]
            print()
            print('deletable_row_position',deletable_row_position)
            
            if(deletable_row_position.shape[0]==0):
                #going_nowhere = True
                current_loop = torch.tensor([])
                break

            deletable_row_position = deletable_row_position[0]
            
            print('Does my edge to delete contain my edge left index ? ', my_edge[0][0] in edge_to_delete)
            print()
            actual_new_test = torch.cat((actual_new_test[:deletable_row_position], actual_new_test[deletable_row_position+1:]))
            if(my_edge[0][0] in edge_to_delete):

                deletable_edge_position1 = (actual_new_test_an == edge_to_delete[0]).nonzero(as_tuple=False)
                deletable_edge_position2 = (actual_new_test_an == edge_to_delete[1]).nonzero(as_tuple=False)

                deletable_edge_position1 = deletable_edge_position1[:,0]

                deletable_edge_position2 = deletable_edge_position2[:,0]

                a_cat_b1, counts1 = torch.cat([deletable_edge_position1, deletable_edge_position2]).unique(return_counts=True)
                deletable_row_position = a_cat_b1[torch.where(counts1.gt(1))]
                print()
                print('deletable_row_position',deletable_row_position)

                if(deletable_row_position.shape[0]==0):
                    #going_nowhere = True
                    current_loop = torch.tensor([])
                    break

                deletable_row_position = deletable_row_position[0]
                
                actual_new_test_an = torch.cat((actual_new_test_an[:deletable_row_position], actual_new_test_an[deletable_row_position+1:]))    
                actual_new_test = actual_new_test_an
                
            #actual_new_test = torch.cat((actual_new_test[:deletable_row_position], actual_new_test[deletable_row_position+1:]))
            #print('what is this', actual_new_test)
            new_test = actual_new_test

            positions1 = (new_test == left_ind).nonzero(as_tuple=False)
            #print('whats happening here',positions1.shape )
            #print('is the same edge still to delete', edge_to_delete)
            if(tracker ==0):
                break

            '''if(positions1.shape[0]>1):
            edge_to_delete = new_test[positions1[0][0]]
            no_branches_formed = False'''
        else:
            first_position = positions1[0][0]
            #print('first_position',first_position)
            adj_edge1 = new_test[positions1[0][0]]
            other_end1 = abs(positions1 - torch.tensor([[0, 1]]))


            consec_pt1 = new_test[other_end1[0][0]][other_end1[0][1]]
            consec_pt1 = int(consec_pt1)
            #print('consec_pt1',consec_pt1)
            print('adj_edge1',adj_edge1)
            current_loop = torch.cat((current_loop,adj_edge1),0)
            current_loop1 = current_loop.reshape(int(current_loop.shape[0]/2),2)
            
            if(consec_pt1 == right_ind):
                #found_right_ind = True
                print('current_loop',current_loop1)
                #current_loop1 = torch.unsqueeze(current_loop1,0)
                print('current_loop shape now',current_loop1.shape)
                print('loop_tracker', loop_tracker)
                loops_collec.append(current_loop1)
                #loops_collec[loop_tracker] = current_loop1
                loop_tracker = loop_tracker + 1
                print()
                print("Wow! Found a loop here")
                print()
                #break

            #else:

            left_ind = consec_pt1
            new_test = torch.cat((new_test[:first_position], new_test[first_position+1:]))
            #print('new_test',new_test)
            tracker = tracker+1
    
    return loops_collec



In [222]:
#testing the function
current_edge_collec = torch.tensor([[10., 11.],
        [4., 15.],
        [14., 15.],
        [11., 12.],
        [2., 3.],
        [ 5., 14.],
        [11., 9.],
        [ 12., 13.],
        [7., 8.],
        [ 3., 4.],
        [9., 2.],
        [13., 5.],
        [8., 4.],
        [ 1.,  2.],
        [1., 5.]])
#my_edge = torch.tensor([[ 9, 18]])

curr_edge = torch.tensor([[ 5, 7]])


In [223]:
expecting_a_cycle(current_edge_collec, curr_edge)

positions1.shape[0] 3
edge_to_delete first tensor([ 5., 14.])
adj_edge1 tensor([ 5., 14.])
positions1.shape[0] 1
adj_edge1 tensor([14., 15.])
positions1.shape[0] 1
adj_edge1 tensor([ 4., 15.])
positions1.shape[0] 2
edge_to_delete first tensor([3., 4.])
adj_edge1 tensor([3., 4.])
positions1.shape[0] 1
adj_edge1 tensor([2., 3.])
positions1.shape[0] 2
edge_to_delete first tensor([9., 2.])
adj_edge1 tensor([9., 2.])
positions1.shape[0] 1
adj_edge1 tensor([11.,  9.])
positions1.shape[0] 2
edge_to_delete first tensor([10., 11.])
adj_edge1 tensor([10., 11.])
positions1.shape[0] 0

deletable_row_position tensor([0])
Does my edge to delete contain my edge left index ?  False

positions1.shape[0] 3
edge_to_delete first tensor([ 5., 14.])
adj_edge1 tensor([ 5., 14.])
positions1.shape[0] 1
adj_edge1 tensor([14., 15.])
positions1.shape[0] 1
adj_edge1 tensor([ 4., 15.])
positions1.shape[0] 2
edge_to_delete first tensor([3., 4.])
adj_edge1 tensor([3., 4.])
positions1.shape[0] 1
adj_edge1 tensor([2., 

[tensor([[ 5., 14.],
         [14., 15.],
         [ 4., 15.],
         [ 8.,  4.],
         [ 7.,  8.]]),
 tensor([[13.,  5.],
         [12., 13.],
         [11., 12.],
         [11.,  9.],
         [ 9.,  2.],
         [ 2.,  3.],
         [ 3.,  4.],
         [ 8.,  4.],
         [ 7.,  8.]]),
 tensor([[1., 5.],
         [1., 2.],
         [2., 3.],
         [3., 4.],
         [8., 4.],
         [7., 8.]])]

In [210]:
a = torch.tensor([[1,2],[5,3],[4,2]])

b = torch.tensor([[1,2],[5,3]])
print(a.shape)
anb = [a,b]

torch.Size([3, 2])


In [211]:
ab

[tensor([[1, 2],
         [5, 3],
         [4, 2]]),
 tensor([[1, 2],
         [5, 3]])]